# Import a survey file from S3

In [1]:
!pip install --upgrade pip boto3

  Using cached pip-22.0.2-py3-none-any.whl (2.1 MB)
  Using cached boto3-1.20.47-py3-none-any.whl (131 kB)
  Attempting uninstall: pip
    Found existing installation: pip 21.3.1
    Uninstalling pip-21.3.1:
      Successfully uninstalled pip-21.3.1


In [2]:
import io
import os

import boto3
import citric
from IPython.display import HTML

In [3]:
citric.__version__

'0.0.8'

In [4]:
LS_URL = os.environ["LIMESURVEY_URL"]
LS_USERNAME = os.environ["LIMESURVEY_USERNAME"]
LS_PASSWORD = os.environ["LIMESURVEY_PASSWORD"]

## Move survey file to S3

- Setup S3 client
- Create a new bucket
- Upload file from local path

In [5]:
s3 = boto3.client("s3", endpoint_url="http://storage:9000")

s3.create_bucket(Bucket="testing")
s3.upload_file("../examples/survey.lss", "testing", "survey.lss")

## Upload survey from S3

- Start `citric` client
- Download file from S3
- Import survey from S3 file
- Display survey questions

In [6]:
with citric.Client(LS_URL, LS_USERNAME, LS_PASSWORD) as client:
    file_object = io.BytesIO()
    s3.download_fileobj("testing", "survey.lss", file_object)

    file_object.seek(0)
    survey_id = client.import_survey(file_object)
    questions = client.list_questions(survey_id)

    for question in questions:
        display(HTML(question["question"]))
        display(HTML(question["help"]))

## Cleanup

- Delete bucket
- Delete survey

In [7]:
s3.delete_object(Bucket="testing", Key="survey.lss")
s3.delete_bucket(Bucket="testing")

with citric.Client(LS_URL, LS_USERNAME, LS_PASSWORD) as client:
    client.delete_survey(survey_id)